In [4]:
import apache_beam as beam
import google.auth
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam import window
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam import window
from apache_beam.transforms.combiners import CountCombineFn
from apache_beam.runners.interactive.interactive_beam import *
import os
# from apache_beam.io import ReadFromBigQuery

# from apitools.backbonemodelgenerator import *
# from apache_beam.io.gcp.internal.clients.storage.storage_v1_client import *
# from apache_beam.io.gcp.internal.clients.storage.storage_v1_messages import *
# beam.io.gcp.internal.clients.bigquery.BigqueryV2()


In [6]:
os.environ["GOOGLE_CLOUD_PROJECT"] = 'fine-glow-364917'

options = pipeline_options.PipelineOptions(flags=[])

credentials, project_id= google.auth.default()

_, options.view_as(GoogleCloudOptions).project = google.auth.default()
options.view_as(GoogleCloudOptions).region = 'europe-west9'
# options.view_as(WorkerOptions).zone = "europe-west1-b"
gcs_location = "gs://flow-beam"
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(gcs_location)

In [18]:
def human_readable_window(window) -> str:
    """Formats a window object into human readable string"""
    if isinstance(window, beam.window.GlobalWindow):
        return str(window)
    return f'{window.start.to_utc_datetime()} - {window.end.to_utc_datetime()}'


@beam.ptransform_fn
def PrintWindowInfo(pcollection):
    """ Prints the window information of number of observations fall under that window."""
    
    class PrintCountsInfo(beam.DoFn):
        def process(self, num_elements, window=beam.DoFn.WindowParam):
            print(f'>> Window[{human_readable_window(window)}] has {num_elements} elements')
            yield num_elements

    return (
        pcollection
        | 'Count elements per window' >> beam.combiners.Count.Globally().without_defaults()
        | 'Print counts info' >> beam.ParDo(PrintCountsInfo())
    )
            
            
            

In [22]:
# sql_query = "SELECT distinct rental_id, start_date \
#          FROM `bigquery-public-data.london_bicycles.cycle_hire` \
#          WHERE Date(start_date) = '2017-01-01' \
#          ORDER BY start_date \
#          LIMIT 100"

sql_query = "SELECT distinct rental_id, start_date \
         FROM `bigquery-public-data.london_bicycles.cycle_hire` \
         WHERE Date(start_date) = '2017-01-01' \
         LIMIT 100"

In [23]:
p = beam.Pipeline(InteractiveRunner(), options=options)

table_row = (p | 'ReadTable' >> beam.io.ReadFromBigQuery(query=sql_query, use_standard_sql=True)
#                | beam.Map(print)
               | 'RentsData' >> beam.Map(lambda x:{'rental_id':x['rental_id'], 'start_date':x['start_date']})
               | 'With Timestamps' >> beam.Map(lambda bikes: beam.window.TimestampedValue(bikes, bikes['start_date'].timestamp()))
#                | beam.Map(print)
            )

# rents_window = table_row | 'window' >> beam.WindowInto(window.FixedWindows(5*60))
# rents_window = table_row | 'window' >> beam.WindowInto(window.SlidingWindows(300, 150))
rents_window = table_row | 'window' >> beam.WindowInto(window.Sessions(30*60))


printWindowinfo = rents_window | 'Print window info' >> PrintWindowInfo()

ib.show(printWindowinfo)




>> Window[2017-01-01 03:17:00 - 2017-01-01 04:39:00] has 9 elements
>> Window[2017-01-01 10:44:00 - 2017-01-01 14:05:00] has 30 elements
>> Window[2017-01-01 00:06:00 - 2017-01-01 02:06:00] has 25 elements
>> Window[2017-01-01 17:19:00 - 2017-01-01 19:05:00] has 7 elements
>> Window[2017-01-01 15:35:00 - 2017-01-01 17:05:00] has 4 elements
>> Window[2017-01-01 04:52:00 - 2017-01-01 05:40:00] has 2 elements
>> Window[2017-01-01 08:27:00 - 2017-01-01 09:20:00] has 4 elements
>> Window[2017-01-01 02:07:00 - 2017-01-01 03:16:00] has 5 elements
>> Window[2017-01-01 14:58:00 - 2017-01-01 15:28:00] has 1 elements
>> Window[2017-01-01 19:30:00 - 2017-01-01 21:08:00] has 7 elements
>> Window[2017-01-01 05:53:00 - 2017-01-01 06:27:00] has 2 elements
>> Window[2017-01-01 07:07:00 - 2017-01-01 07:37:00] has 1 elements
>> Window[2017-01-01 10:11:00 - 2017-01-01 10:41:00] has 1 elements
>> Window[2017-01-01 21:32:00 - 2017-01-01 22:02:00] has 2 elements
